In [1]:
import os
import csv
import gym
import glob
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

from stable_baselines import A2C
from stable_baselines import DQN

import tensorflow as tf

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class IdsEnv(gym.Env):
    def __init__(self, dataset, images_per_episode=1, random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(dataset[0].columns) - 1,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

In [3]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]

def mal_ben(df, label, normal):
    print("Preprocessing: make labels binary")
    print()
    df[label] = df[label].astype('object')
    atk_idx = df.loc[df[label] != normal].index
    df.loc[atk_idx, label] = 1.0
    df.loc[df.index.difference(atk_idx), label] = 0.0
    df[label] = df[label].astype(dtype=np.float32)

def preproc(df_train, df_test, columns):
    print("Preprocessing: One hot encoding + abundant features")
    print()
    df_train = pd.get_dummies(df_train,columns=columns)
    df_test = pd.get_dummies(df_test,columns=columns)
    extra_removables = df_test.columns ^ df_train.columns
    print(extra_removables)
    for to_remove in extra_removables:
            try:
                df_test.drop(to_remove, inplace=True, axis=1)
            except:
                print(f"{to_remove} already not part of test set, skipping")
            try:
                df_train.drop(to_remove, inplace=True, axis=1)
            except:
                print(f"{to_remove} already not part of train set, skipping")
    return df_train, df_test

def split_df(df, d):
    print("Preprocessing: split into X and Y sets")
    print()
    if d == 'nslkdd':
        label = 'class'
        col = label
        print(col)
        cols = df.loc[:, df.columns != label].columns
        print(cols)
    elif d == 'unswnb15':
        label = 'attack_cat'
        col = label
        print(col)
        cols = df.loc[:, df.columns != label].columns
        print(cols)
    else:
        print(f"{d} not a valid dataset")
    return df[cols], df[col]

def top_feat(df, dataset):
    print("Preprocessing: convert to top features")
    print()
    features = {
        'cicdos2017': ["Init Bwd Win Bytes",  "Idle Min", "ACK Flag Count", "Fwd Packet Length Min", "Fwd PSH Flags"],
        'cicids2017': ["Avg Packet Size", "Packet Length Variance", "Fwd Packet Length Mean", "Init Fwd Win Bytes", "Bwd IAT Total"],
        'cicddos2019': ["URG Flag Count", "Down/Up Ratio", "Bwd Packet Length Min", "ACK Flag Count", "Fwd Packets Length Total"],
        'nslkdd': ["dst_host_serror_rate", "service_private", "count","dst_host_count", "service_domain_u", "flag_REJ", "dst_host_diff_srv_rate"],
        'unsw': ["sttl", "dttl", "ct_state_ttl", "service", "dload", "rate", "dmean", "dbytes", "dur", "is_sm_ips_ports", "dloss"],
    }
    important_features = features[dataset] + ["Label"] # Adding class for custom environment logic
    important_features = list(set(important_features))
    removable_features = df.columns ^ important_features
    
    return df.drop(labels=removable_features, axis='columns')

def ids_eval(model, x_test, y_test):
    print("Evaluation: starting validation of the model")
    print()
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        try:
            FPR = (float(FP) / (TN + FP)) # 1 - specificity
        except:
            FPR = 0.0
        f1_score = 2 * (precision * recall) / (precision + recall)
        print()
        print('Evaluation: validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]


In [4]:
generalization_sets = ['cicddos2019', 'cicdos2017', 'cicids2017']
generalization_folders = ['cic-ddos2019', 'cic-dos2017', 'cic-ids2017']
datasets = ['nslkdd' , 'cicddos2019' , 'cicdos2017'] # 'cicids2017', 'cicddos2019-top', 'unswnb15', 

end_d = ['model', 'train dataset', 'test dataset', 'features', 'accuracy', 'precision', 'recall', 'FPR', 'F1 score', 'notes']
end_df = pd.DataFrame(columns = end_d)

start_time = time.time()
for d in datasets:
    dd = f'/project/normalization-runs/models/{d}/DQN'
    ad = f'/project/normalization-runs/models/{d}/A2C'
    os.chdir(dd)
    print("===========================")
    print("Evaluating DQN based models")
    print()
    for m in glob.glob("*.pkl"):
        print(f"Model {m}")
        print()
        print(f'Dataset {d}')
        print("===========================")
        model = DQN.load(f'{m}') 
        if d == 'nslkdd':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTrain.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTest.feather')
            mal_ben(df_train, 'class', 'normal')
            mal_ben(df_test, 'class', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['protocol_type' ,'service', 'flag'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df_test, d)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            ids_time = time.time()
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'nslkdd', 'nslkdd', f'{len(df.columns)}'] + results + ['']
            print("===========================")
            print(entry)
            print("===========================")
            total_duration = time.time() - start_time
            eval_duration = time.time() - ids_time
            print(f"Total duration: {total_duration}")
            print(f"Evaluation duration: {eval_duration}")
            print("===========================")
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'unswnb15':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_training-set.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_testing-set.feather')
            df_train.drop('label', inplace=True, axis=1)
            df_test.drop('label', inplace=True, axis=1)
            mal_ben(df_train, 'attack_cat', 'normal')
            mal_ben(df_test, 'attack_cat', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['proto' ,'service', 'state'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df, d)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            ids_time = time.time()
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'unswnb15', 'unswnb15', f'{len(df.columns)}'] + results + ['']
            print("===========================")
            print(entry)
            print("===========================")
            total_duration = time.time() - start_time
            eval_duration = time.time() - ids_time
            print(f"Total duration: {total_duration}")
            print(f"Evaluation duration: {eval_duration}")
            print("===========================")
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'cicddos2019':
            for i, x in enumerate(generalization_sets):
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry
        if d == 'cicddos2019-top':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicddos2019')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry
        if d == 'cicdos2017-top':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicdos2017')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicdos2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
        if d == 'cicids2017-top':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicids2017')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicids2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry 
        if d == 'cicdos2017':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicdos2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry 
        if d == 'cicids2017':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicids2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
    os.chdir(ad)
    for m in glob.glob("*.pkl"):
        model = A2C.load(f'{m}') 
        if d == 'nslkdd':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTrain.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTest.feather')
            mal_ben(df_train, 'class', 'normal')
            mal_ben(df_test, 'class', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['protocol_type' ,'service', 'flag'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df_test, d)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            ids_time = time.time()
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'nslkdd', 'nslkdd', f'{len(df.columns)}'] + results + ['']
            print("===========================")
            print(entry)
            print("===========================")
            total_duration = time.time() - start_time
            eval_duration = time.time() - ids_time
            print(f"Total duration: {total_duration}")
            print(f"Evaluation duration: {eval_duration}")
            print("===========================")
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'unswnb15':
            df_train = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_training-set.feather')
            df_test = pd.read_feather('/project/datasets/clean-ids-collection/unsw-nb15/clean/designated-train-test-sets/UNSW_NB15_testing-set.feather')
            df_train.drop('label', inplace=True, axis=1)
            df_test.drop('label', inplace=True, axis=1)
            mal_ben(df_train, 'attack_cat', 'normal')
            mal_ben(df_test, 'attack_cat', 'normal')
            df_train, df_test = preproc(df_train, df_test, ['proto' ,'service', 'state'])
            df = pd.concat([df_train, df_test], ignore_index=True)
            x_test, y_test = split_df(df, d)
            y_test = y_test.to_frame()
            x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
            y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
            ids_time = time.time()
            results = ids_eval(model, x_test, y_test)
            entry = [f'{m}', 'unswnb15', 'unswnb15', f'{len(df.columns)}'] + results + ['']
            print("===========================")
            print(entry)
            print("===========================")
            total_duration = time.time() - start_time
            eval_duration = time.time() - ids_time
            print(f"Total duration: {total_duration}")
            print(f"Evaluation duration: {eval_duration}")
            print("===========================")
            df_length = len(end_df)
            end_df.loc[df_length] = entry
        if d == 'cicddos2019':
            for i, x in enumerate(generalization_sets):
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry
        if d == 'cicddos2019-top':
            for i, x in enumerate(generalization_sets):
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicddos2019')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicddos2019', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
        if d == 'cicdos2017-top':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicdos2017')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicdos2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry 
        if d == 'cicids2017-top':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                df = top_feat(df, 'cicids2017')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicids2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
        if d == 'cicdos2017':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicdos2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry 
        if d == 'cicids2017':
            for i, x in enumerate(generalization_sets):
                print(x)
                df = pd.read_feather(f'/project/datasets/clean-ids-collection/{generalization_folders[i]}/clean/{x}.feather')
                mal_ben(df, 'Label', 'Benign')
                x_train, y_train, x_val, y_val, x_test, y_test  = balancing_dataset_sampler_df(df, train_frac=0.0, val_frac=0.0, test_frac=1.0)
                y_test = y_test.to_frame()
                x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
                y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])
                ids_time = time.time()
                results = ids_eval(model, x_test, y_test)
                entry = [f'{m}', 'cicids2017', x, f'{len(df.columns)}'] + results + ['']
                print("===========================")
                print(entry)
                print("===========================")
                total_duration = time.time() - start_time
                eval_duration = time.time() - ids_time
                print(f"Total duration: {total_duration}")
                print(f"Evaluation duration: {eval_duration}")
                print("===========================")
                df_length = len(end_df)
                end_df.loc[df_length] = entry  
    end_df.to_csv(f'/project/normalization-runs/normalization-results-{d}.csv', index=False) 

            
end_df.info()
end_df.describe()

end_df.to_csv('/project/normalization-runs/normalization-results.csv', index=False)

Evaluating DQN based models

Model per_dd_DQN_1.pkl

Dataset nslkdd
Loading a model without an environment, this model cannot be trained until it has a valid environment.






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Preprocessing: make labels binary

Preprocessing: make labels binary

Preprocessing: One hot encoding + abundant features

Index(['service_aol', 'service_harvest', 'service_http_2784',
       'service_http_8001', 'service_red_i', 'service_urh_i'],
      dtype='object')
service_aol already not part of test set, skipping
service_harvest already not part of test set, skipping
service_http_2784 already not part of test set, skipping
service_http_8001 already not part of test set, skipping
service_red_i already not part of test set, skipping
service_urh_i already not part of test set, skipping
Preprocessing: split into X and Y sets

class
Index(['duration', 'src_

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



Evaluation: validation done...
Accuracy: 75.76967438559134%
Precision: 93.12422102201911%
Recall/TPR/Sensitivity: 65.13148336481186%
FPR: 7.543299908842298%
F1 score: 0.7665213302556211
['per_dd_DQN_1.pkl', 'nslkdd', 'nslkdd', '117', 0.7576967438559135, 0.9312422102201912, 0.6513148336481186, 0.07543299908842298, 0.7665213302556211, '']
Total duration: 29.95555019378662
Evaluation duration: 22.020174026489258
Model per_dd_DQN_2.pkl

Dataset nslkdd
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Preprocessing: make labels binary

Preprocessing: make labels binary

Preprocessing: One hot encoding + abundant features

Index(['service_aol', 'service_harvest', 'service_http_2784',
       'service_http_8001', 'service_red_i', 'service_urh_i'],
      dtype='object')
service_aol already not part of test set, skipping
service_harvest already not part of test set, skipping
service_http_2784 already not part of test set, skipping
service_htt

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



Evaluation: validation done...
Accuracy: 79.40732854227663%
Precision: 96.84254258412962%
Recall/TPR/Sensitivity: 68.24769433465086%
FPR: 3.423423423423423%
F1 score: 0.800686990124517
['per_dd_DQN_2.pkl', 'nslkdd', 'nslkdd', '117', 0.7940732854227663, 0.9684254258412962, 0.6824769433465085, 0.03423423423423423, 0.800686990124517, '']
Total duration: 59.48328185081482
Evaluation duration: 22.021538972854614
Model per_dd_DQN_3.pkl

Dataset nslkdd
Loading a model without an environment, this model cannot be trained until it has a valid environment.
Preprocessing: make labels binary

Preprocessing: make labels binary

Preprocessing: One hot encoding + abundant features

Index(['service_aol', 'service_harvest', 'service_http_2784',
       'service_http_8001', 'service_red_i', 'service_urh_i'],
      dtype='object')
service_aol already not part of test set, skipping
service_harvest already not part of test set, skipping
service_http_2784 already not part of test set, skipping
service_http_

...


Evaluation: validation done...
Accuracy: 64.06533998229015%
Precision: 81.61439157384537%
Recall/TPR/Sensitivity: 60.41120463640127%
FPR: 28.328306764325724%
F1 score: 0.6943007235603132
['default_A2C_2.pkl', 'cicdos2017', 'cicddos2019', '67', 0.6406533998229016, 0.8161439157384537, 0.6041120463640127, 0.2832830676432572, 0.6943007235603132, '']
Total duration: 46495.64321923256
Evaluation duration: 74.86992979049683
cicdos2017
Preprocessing: make labels binary

Label
Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Fwd Packets Length Total', 'Bwd Packets Length Total',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
 

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



Evaluation: validation done...
Accuracy: 57.10024700563918%
Precision: 82.24355688120427%
Recall/TPR/Sensitivity: 54.72447049337923%
FPR: 35.717633823943004%
F1 score: 0.6571949947862357
['default_A2C_3.pkl', 'cicdos2017', 'cicddos2019', '67', 0.5710024700563918, 0.8224355688120427, 0.5472447049337923, 0.35717633823943, 0.6571949947862357, '']
Total duration: 47192.02976465225
Evaluation duration: 74.78996205329895
cicdos2017
Preprocessing: make labels binary

Label
Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Fwd Packets Length Total', 'Bwd Packets Length Total',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
   

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



Evaluation: validation done...
Accuracy: 51.029966910565314%
Precision: 83.50654797968029%
Recall/TPR/Sensitivity: 50.62439961575408%
FPR: 47.06117021276596%
F1 score: 0.6303495101229528
['default_A2C_4.pkl', 'cicdos2017', 'cicddos2019', '67', 0.5102996691056532, 0.8350654797968029, 0.5062439961575408, 0.47061170212765957, 0.6303495101229528, '']
Total duration: 47887.89866518974
Evaluation duration: 74.72202181816101
cicdos2017
Preprocessing: make labels binary

Label
Index(['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
       'Fwd Packets Length Total', 'Bwd Packets Length Total',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
